In [88]:
import numpy as np
import random
import math

In [133]:
class MLP:
    #构造函数 初始化模型参数
    def __init__(self, i_num, h_num, n_num, o_num):
        self.learn_rate = 0.1    #学习率

        #输入参数
        self.input_layer_nerual_num = i_num    #输入层神经元 数量
        self.hidden_layers_num = h_num                #隐层 数量
        self.hidden_layers_neural_num = n_num  #每个隐藏层的神经元数量
        self.output_layer_nerual_num = o_num   #输出层神经元数量
        
        #模型参数
        self.input_layer_value = []                  #输入层
        self.hidden_layer_value = []                 #隐层
        self.hidden_layer_value_without_bias = []    #无bias的隐层
        self.weights = []                            #权重矩阵
        self.weights_without_bias = []               #无bias的权重矩阵
        self.output_layer_value = []                 #输出层
        self.output_layer_softmax = []               #输出层softmax
        self.output_expectation = []                 #输出期望值
        self.cost = []                               #误差
    
    # 输入数据
    def input_params(self, data, output):
        self.input_layer_value = []
        data.insert(0, 1)
        for value in data:
            self.input_layer_value.append(value)
        for v in output:
            self.output_expectation.append(v)
    
    # 初始化权值矩阵（加入bias）
    def _ini_weight(self):
        x = self.input_layer_nerual_num
        y = self.hidden_layers_num
        hidden_neruals = self.hidden_layers_neural_num
        o = self.output_layer_nerual_num
        
        self.weights = []
        
        #输入层和隐藏层的权值
        weights__1 = []
        for i in range(0, hidden_neruals[0]):
            weights_1 = []
            for j in range(0, x + 1):
                weight = random.uniform(-1, 1)
                weights_1.append(weight)
            weights__1.append(weights_1)
        self.weights.append(weights__1)
        
        #隐藏层之间的权值
        for q in range(0, y-1):
            weights__2 = []
            for i in range(0, hidden_neruals[q+1]):
                weights_1 = []
                for j in range(0, hidden_neruals[q] + 1):
                    weight = random.uniform(-1, 1)
                    weights_1.append(weight)
                weights__2.append(weights_1)
            self.weights.append(weights__2)
        
        #隐藏层和输出层的权值
        weights__3 = []
        for i in range(0, o):
            weights_1 = []
            for j in range(0, hidden_neruals[y-1] + 1):
                weight = random.uniform(-1, 1)
                weights_1.append(weight)
            weights__3.append(weights_1)   
        self.weights.append(weights__3)
        self.cal_weight_without_bias()
    
    # 计算没有bias的权值矩阵（将原始权值矩阵去掉第一列）
    def cal_weight_without_bias(self):
        for i in self.weights:
            weights2 = []
            for j in i:
                weights2.append(j[1:])
            self.weights_without_bias.append(weights2)
    
    # Sigmoid激活函数
    def sigmoid(self, value):
        sig = 1/(1+np.exp(-1*value))
        return sig
    
    def sigmoid_derivative(self, value):
        sig_value = self.sigmoid(value)
        return sig_value * (1 - sig_value)

    
    # 前向传播
    def feed_forward(self):
        # 将自身元素置零，避免循环使用时出错
        self.hidden_layer_value = []
        self.hidden_layer_value_without_bias = []
        self.output_layer_value = []
        self.output_layer_softmax = []
        # 输入层到隐藏层
        X0 = np.array(self.input_layer_value)
        W0 = np.array(self.weights[0])
        
        # 计算求和，sigmoid函数计算，添加bias
        self.hidden_layer_value.append(np.dot(X0,W0.T).tolist())
        for i, j in enumerate(self.hidden_layer_value[0]):
            j = self.sigmoid(j)
            self.hidden_layer_value[0][i] = j
        
        self.hidden_layer_value[0].insert(0, 1)

        print('NUM1')
        print("X0:" + str(X0) + '\n' + 'W0:' + str(W0) + '\n')
        print(self.hidden_layer_value)    
        
        # 隐藏层之间到输出层
        for i in range(0, self.hidden_layers_num):
            W = np.array(self.weights[i+1])
            X = np.array(self.hidden_layer_value[i])
            
            print('NUM' + str(i + 2))
            print("X:" + str(X) + '\n' + 'W:' + str(W) + '\n')
            
            
            self.hidden_layer_value.append(np.dot(X,W.T).tolist())
            
            for k, j in enumerate(self.hidden_layer_value[i + 1]):
                j = self.sigmoid(j)
                self.hidden_layer_value[i + 1][k] = j
            if i != self.hidden_layers_num - 1:
                self.hidden_layer_value[i + 1].insert(0, 1)
        self.output_layer_value = self.hidden_layer_value[self.hidden_layers_num]
        print("hidden layer value:" + str(self.hidden_layer_value))
        # 计算输出层softmax
        self.Softmax_function()
        print("softmax value:" + str(self.output_layer_softmax))
        
        # 计算去除bias的隐藏层
        self.cal_hidden_layer()
    
    def cal_hidden_layer(self):
        for i in self.hidden_layer_value:
            self.hidden_layer_value_without_bias.append(i[1:])
        self.hidden_layer_value_without_bias.pop()
   
    # Softmax函数
    def Softmax_function(self):
        sum = 0
        soft = []
        for value in self.output_layer_value:
            sum += np.exp(value)
        for value in self.output_layer_value:
            softmax_value = np.exp(value)
            soft.append(softmax_value / sum)
        self.output_layer_softmax = soft
    
    # 反向传播
    def back_propagation(self):
        self.calculate_cost()
        self.back_update_weights()
        self.update_weights()
    
    # 反向更新权重矩阵（不含bias）
    def back_update_weights(self):
        input_value = self.input_layer_value[1:]
        self.hidden_layer_value_without_bias.insert(0, input_value)
        
        for i in range(0,self.hidden_layers_num + 1):
            print("backpropagation" + str(i + 1))
            
            m1 = np.array(self.cost[i])
            m1 = m1.reshape(m1.shape[0], 1)
            m2 = np.array(self.hidden_layer_value_without_bias[i])
            m2 = m2.reshape(1, m2.shape[0])
            print(m1.shape)
            print(m2.shape)
            
            self.weights_without_bias[i] = self.weights_without_bias[i] - self.learn_rate * np.dot(m1, m2)
    
    # 计算cost矩阵
    def calculate_cost(self):
        # 初始化cost矩阵，避免循环时出错
        self.cost = []
        
        # 输出层到隐藏层
        output_layer_value = np.array(self.output_layer_softmax)
        output_expectation = np.array(self.output_expectation)
                
        cost =  output_layer_value - output_expectation
        self.cost.append(cost.tolist())
        
        
        # 隐藏层之间
        for i in range(0, self.hidden_layers_num):
            de = np.array(self.hidden_layer_value_without_bias[self.hidden_layers_num - i - 1])
            cost = np.dot(self.cost[i], self.weights_without_bias[self.hidden_layers_num - i]) * self.sigmoid_derivative(de)
            self.cost.append(cost.tolist())
        
        self.cost = self.cost[::-1]
        print('cost matrix:' + str(self.cost))
    
    # 更新权重矩阵（将不加bias的矩阵更新到加bias的矩阵上面）
    def update_weights(self):
        for i in range(0, self.hidden_layers_num + 1):
            for j in range(0, len(self.weights_without_bias[i])):
                for k in range(0, len(self.weights_without_bias[i][j])):
                    self.weights[i][j][k + 1] = self.weights_without_bias[i][j][k]
     
    
    def run(self, times):
        for i in range(0, times):
            self.feed_forward()
            self.back_propagation()
            
#     def get_weight(self):
#         return self.weights
    
#     def get_weights2(self):
#         return self.weights_without_bias
    
#     def get_input_layer(self):
#         return self.input_layer_value
    
#     def get_hidden_layer(self):
#         return self.hidden_layer_value
    
#     def get_hidden_layer_2(self):
#         return self.hidden_layer_value_without_bias
    
    
#     def cost_matrix(self):
#         return self.cost
        
#     def get_softmax(self):
#         return self.output_layer_softmax
    


In [134]:
mlp = MLP(4, 3, [5, 4, 3], 2)

In [135]:
mlp.input_params([1, 2, 3, 4], [0.9, 0.1])

In [136]:
mlp._ini_weight()

In [137]:
mlp.run(100)

NUM1
X0:[1 1 2 3 4]
W0:[[ 0.06603991  0.83420057  0.20811918  0.06779417  0.27579916]
 [-0.26498344  0.02453467 -0.55115316 -0.08524683  0.34586447]
 [-0.723215    0.69021624  0.32198003  0.71066098  0.98163827]
 [ 0.76803778 -0.08227462 -0.26214752 -0.70924785 -0.40659483]
 [ 0.8324843   0.91965742 -0.00882449  0.55312768 -0.96985604]]

[[1, 0.9323308892881144, 0.4464467767940145, 0.9987325719359541, 0.026787822198923606, 0.38094281468331403]]
NUM2
X:[1.         0.93233089 0.44644678 0.99873257 0.02678782 0.38094281]
W:[[-0.9119862  -0.7384232   0.32851469  0.25246493  0.04654333 -0.1556214 ]
 [-0.36163438 -0.34624749  0.68487164  0.40238407 -0.80668485  0.38221352]
 [ 0.12781279 -0.66379368  0.0856583  -0.1846021  -0.5623278  -0.7672246 ]
 [-0.73953453  0.99414972 -0.49862768  0.1936547   0.53703521  0.49040274]]

NUM3
X:[1.         0.22103109 0.53672707 0.27998802 0.58888112]
W:[[-0.22319567  0.52691046  0.90651287  0.05589469  0.9550273 ]
 [ 0.76032857 -0.39778321  0.21635528  0.24

X:[1.         0.21149372 0.65737548 0.29406038 0.68859312]
W:[[-0.22319567  0.60548691  1.12681838  0.16458558  1.19470044]
 [ 0.76032857 -0.23044423  0.67668414  0.46986329  0.6249467 ]
 [-0.1887346   0.69507438  0.55395539 -0.25265961 -0.13325652]]

NUM4
X:[1.         0.82004677 0.84876261 0.53901285]
W:[[-0.55635975  1.66541909  2.00271465  0.07201126]
 [-0.24352721 -0.67291301 -1.75926491 -0.55513446]]

hidden layer value:[[1, 0.9585936635857387, 0.52731725176672, 0.9993138003569508, 0.01369966753416105, 0.5401776769250073], [1, 0.21149372002636255, 0.6573754846942881, 0.2940603804986276, 0.6885931186178491], [1, 0.8200467698097681, 0.8487626084171508, 0.5390128526136764], [0.9274394483295975, 0.06992966618398698]]
softmax value:[0.7021401140356927, 0.29785988596430724]
cost matrix:[[-0.004745430507322925, -0.005997917321132844, -0.0091184049501436, 0.006847761945125616, -0.007839442040672464], [-0.010762425806177157, -0.05224223566562625, -0.020129451162931168, -0.0469848094129451

hidden layer value:[[1, 0.9861489861556365, 0.7983058212415995, 0.9999011724342032, 0.004746161278041233, 0.8407345228530936], [1, 0.256994044809249, 0.8638525140773454, 0.3217651774670681, 0.838426622475868], [1, 0.9314553253066661, 0.9486125855581014, 0.6290189651255625], [0.988174695392389, 0.012444132148882119]]
softmax value:[0.7262602458423774, 0.2737397541576226]
cost matrix:[[-0.01695546630368017, -0.017515757678029053, -0.026712659636173086, 0.011530531569716786, -0.01972964293269018], [-0.029191089153450565, -0.09087338642330468, -0.035493133820542665, -0.08333401139015108], [-0.12748224576156017, -0.1775376758401267, -0.05791204874511476], [-0.17373975415762266, 0.17373975415762258]]
backpropagation1
(5, 1)
(1, 4)
backpropagation2
(4, 1)
(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 0.06603991  0.89064249  0.321003    0.23711991  0.50156682]
 [-0.26498344  0.07931214 -0.44159822  0.07908559  0.56497436]
 [-0.723215    0.777972

In [131]:
mlp.output_layer_softmax

[0.6921557848431698, 0.3078442151568302]

In [132]:
mlp.run(1000)

NUM1
X0:[1 1 2 3 4]
W0:[[ 0.10111172 -0.46067327  0.71413166  0.36499989  0.55272368]
 [ 0.82737377 -0.05615722 -0.56216278 -0.59094739 -0.09327266]
 [-0.1829579  -0.18444725  0.15568608  0.42028422  0.7594644 ]
 [ 0.9010089   0.530888   -0.32310028 -0.6076289   0.36474547]
 [ 0.16127608  0.97596783  1.03090203  0.96582851 -0.28871594]]

[[1, 0.9875633898143301, 0.07592537890495239, 0.985834475003497, 0.603911871177262, 0.9929079938004316]]
NUM2
X:[1.         0.98756339 0.07592538 0.98583448 0.60391187 0.99290799]
W:[[ 0.7616617   0.1944657   0.91054364  0.62878335  0.94767021  0.77172215]
 [-0.68227572  0.33525236  0.5929681   1.01131224  0.73225749 -0.45478634]
 [ 0.78601115 -0.86642745 -0.40005402  0.27792715 -0.86888334 -0.87201282]
 [ 0.90483172  0.42243032 -0.1582561   0.73388291  0.14587553  0.85575063]]

NUM3
X:[1.         0.95172108 0.66406739 0.22853844 0.95125861]
W:[[ 0.38998666  1.58304924  0.88252445  0.35972764  0.85079859]
 [-0.50456679 -0.80790744  0.11808123 -0.255591

(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 0.10111172 -0.38415309  0.86717203  0.59456044  0.85880441]
 [ 0.82737377  0.07848787 -0.2928726  -0.18701212  0.4453077 ]
 [-0.1829579   0.02034563  0.56527185  1.03466287  1.57863592]
 [ 0.9010089   0.68696456 -0.01094716 -0.13939921  0.98905172]
 [ 0.16127608  1.09484737  1.26866112  1.32246715  0.18680224]]

[[1, 0.998733463122139, 0.8235099111659232, 0.9999691537256159, 0.9939647734088193, 0.9997982049401459]]
NUM2
X:[1.         0.99873346 0.82350991 0.99996915 0.99396477 0.9997982 ]
W:[[ 0.7616617   0.70229711  1.10984547  1.13834053  1.39536256  1.28153663]
 [-0.68227572  0.5904942   0.6984692   1.2674226   0.95990865 -0.19858286]
 [ 0.78601115 -0.72132638 -0.34330827  0.4235221  -0.74103979 -0.72634375]
 [ 0.90483172  0.70564291 -0.04037187  1.0180607   0.39892751  1.1400249 ]]

NUM3
X:[1.         0.99794118 0.92453508 0.22159855 0.98418225]
W:[[ 0.38998666  2.67395531  1.7707215   0.59731263  1.92534979]
 [-0.50456679 -0.87660858  0.0633

NUM1
X0:[1 1 2 3 4]
W0:[[ 0.10111172 -0.02127505  1.59292811  1.68319456  2.31031657]
 [ 0.82737377  0.44915315  0.44845797  0.92498373  1.92796884]
 [-0.1829579   0.61914898  1.76287854  2.83107291  3.97384931]
 [ 0.9010089   1.16765491  0.95043354  1.30267183  2.91181311]
 [ 0.16127608  1.52103401  2.1210344   2.60102707  1.8915488 ]]

[[1, 0.9999999762704058, 0.9999968252212059, 0.9999999999995131, 0.9999999966849689, 0.9999999994346949]]
NUM2
X:[1.         0.99999998 0.99999683 1.         1.         1.        ]
W:[[ 0.7616617   1.5490555   1.93517615  1.98525049  2.24175096  2.12842877]
 [-0.68227572  1.12830024  1.2236585   1.8053179   1.49749931  0.339302  ]
 [ 0.78601115 -0.47540864 -0.1034466   0.66948269 -0.49522651 -0.48038821]
 [ 0.90483172  1.29720719  0.53710774  1.60972463  0.99025092  1.73167716]]

NUM3
X:[1.         0.99997512 0.99509117 0.47527624 0.99915117]
W:[[ 0.38998666  3.84674501  2.91023417  0.97502421  3.09171937]
 [-0.50456679 -0.90845089  0.03258063 -0.28041

backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 0.10111172  1.50130173  4.63808166  6.2509249   8.40062369]
 [ 0.82737377  1.97006229  3.49027626  5.48771116  8.01160541]
 [-0.1829579   2.62806535  5.78071128  8.85782202 12.00951479]
 [ 0.9010089   2.91026672  4.43565716  6.53050726  9.88226035]
 [ 0.16127608  3.14680085  5.37256807  7.47832758  8.39461615]]

[[1, 1.0, 1.0, 1.0, 1.0, 1.0]]
NUM2
X:[1. 1. 1. 1. 1. 1.]
W:[[ 0.7616617   3.10887757  3.49499802  3.54507257  3.80157304  3.68825084]
 [-0.68227572  2.27625182  2.37160994  2.95326949  2.6454509   1.48725359]
 [ 0.78601115  0.0792945   0.45125648  1.22418583  0.05947664  0.07431494]
 [ 0.90483172  2.51620058  1.75610097  2.82871802  2.20924431  2.95067054]]

NUM3
X:[1.         0.99999999 0.99998414 0.93550746 0.99999808]
W:[[ 0.38998666  5.32578998  4.38772791  2.04814663  4.57051747]
 [-0.50456679 -0.92369705  0.01735696 -0.29071152 -1.47613018]
 [ 0.43950972  1.90801596  0.99505613  0.87210292  1.08776215]]

NUM4
X:[1.

(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 0.10111172  6.51509037 14.66565894 21.29229081 28.45577824]
 [ 0.82737377  6.98742895 13.52500957 20.53981113 28.08107203]
 [-0.1829579   8.52239263 17.56936585 26.54080387 35.58682392]
 [ 0.9010089   8.27856345 15.17225063 22.63539746 31.35544729]
 [ 0.16127608  8.31016003 15.69928644 22.96840513 29.04805289]]

[[1, 1.0, 1.0, 1.0, 1.0, 1.0]]
NUM2
X:[1. 1. 1. 1. 1. 1.]
W:[[ 0.7616617   5.60093053  5.98705098  6.03712553  6.293626    6.1803038 ]
 [-0.68227572  4.28218723  4.37754535  4.9592049   4.65138631  3.493189  ]
 [ 0.78601115  1.28646657  1.65842855  2.43135789  1.2666487   1.281487  ]
 [ 0.90483172  4.60058341  3.84048381  4.91310085  4.29362714  5.03505338]]

NUM3
X:[1.         1.         1.         0.99983516 1.        ]
W:[[ 0.38998666  7.0061377   6.06807225  3.70456207  6.25086481]
 [-0.50456679 -0.92430008  0.01675393 -0.2912451  -1.47673322]
 [ 0.43950972  2.88904026  1.97607853

...

X:[1.        1.        0.9998155 1.       ]
W:[[  0.70126531  15.55027268   3.16181023  12.71727338]
 [  0.30062859 -14.73240804  -3.31688215 -13.17753614]]

hidden layer value:[[1, 1.0, 1.0, 1.0, 1.0, 1.0], [1, 1.0, 1.0, 1.0, 1.0], [1, 1.0, 0.9998154954995246, 1.0], [0.9999999999999889, 3.708418589040495e-14]]
softmax value:[0.7310585786299955, 0.2689414213700046]
cost matrix:[[-3403.8520146612714, -3404.105354443698, -3411.0806870423626, -3405.987968776373, -3404.5115420996617], [-14.77726633693749, -14.47717818518168, -13.973640880379866, -14.465811131907067], [-1.00586647818609, -0.21521394246470815, -0.8601193897606236], [-0.16894142137000456, 0.1689414213700046]]
backpropagation1
(5, 1)
(1, 4)
backpropagation2
(4, 1)
(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 1.01111720e-01  3.56552528e+04  7.13121411e+04  1.06967505e+05
   1.42623407e+05]
 [ 8.27373767e-01  3.56605738e+04  7.13206978e+04  1.06981299e+05
   1.42642427e+05]
 [-1.

backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 1.01111720e-01  5.38181136e+04  1.07637863e+05  1.61456088e+05
   2.15274850e+05]
 [ 8.27373767e-01  5.38246504e+04  1.07648851e+05  1.61473529e+05
   2.15298733e+05]
 [-1.82957904e-01  5.40025005e+04  1.08005525e+05  1.62008475e+05
   2.16011499e+05]
 [ 9.01008904e-01  5.38749098e+04  1.07748435e+05  1.61622529e+05
   2.15497880e+05]
 [ 1.61276080e-01  5.38374522e+04  1.07673983e+05  1.61510394e+05
   2.15345616e+05]]

[[1, 1.0, 1.0, 1.0, 1.0, 1.0]]
NUM2
X:[1. 1. 1. 1. 1. 1.]
W:[[  0.7616617  386.98350718 387.36962762 387.41970217 387.67620265
  387.56288045]
 [ -0.68227572 372.22022719 372.31558531 372.89724485 372.58942626
  371.43122895]
 [  0.78601115 347.91588372 348.2878457  349.06077504 347.89606585
  347.91090415]
 [  0.90483172 373.54374324 372.78364363 373.85626068 373.23678697
  373.9782132 ]]

NUM3
X:[1. 1. 1. 1. 1.]
W:[[ 0.38998666 50.3432285  49.40516304 47.0416166  49.5879556 ]
 [-0.

backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 1.01111720e-01  8.47170213e+04  1.69435678e+05  2.54152811e+05
   3.38870481e+05]
 [ 8.27373767e-01  8.47252355e+04  1.69450021e+05  2.54175284e+05
   3.38901073e+05]
 [-1.82957904e-01  8.49487387e+04  1.69898002e+05  2.54847190e+05
   3.39796452e+05]
 [ 9.01008904e-01  8.47877340e+04  1.69574083e+05  2.54361002e+05
   3.39149177e+05]
 [ 1.61276080e-01  8.47406229e+04  1.69480325e+05  2.54219907e+05
   3.38958299e+05]]

[[1, 1.0, 1.0, 1.0, 1.0, 1.0]]
NUM2
X:[1. 1. 1. 1. 1. 1.]
W:[[  0.7616617  483.22173945 483.6078599  483.65793445 483.91443492
  483.80111273]
 [ -0.68227572 466.85630694 466.95166506 467.53332461 467.22550602
  466.06730871]
 [  0.78601115 439.76272782 440.13468981 440.90761915 439.74290996
  439.75774826]
 [  0.90483172 468.02593466 467.26583506 468.3384521  467.71897839
  468.46040463]]

NUM3
X:[1. 1. 1. 1. 1.]
W:[[ 0.38998666 55.87381158 54.93574613 52.57219969 55.11853869]
 [-0.

backpropagation2
(4, 1)
(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 3 4]
W0:[[ 1.01111720e-01  1.27180953e+05  2.54363541e+05  3.81544605e+05
   5.08726206e+05]
 [ 8.27373767e-01  1.27191040e+05  2.54381631e+05  3.81572699e+05
   5.08764293e+05]
 [-1.82957904e-01  1.27465239e+05  2.54931003e+05  3.82396692e+05
   5.09862455e+05]
 [ 9.01008904e-01  1.27267078e+05  2.54532771e+05  3.81799033e+05
   5.09066552e+05]
 [ 1.61276080e-01  1.27209256e+05  2.54417590e+05  3.81625805e+05
   5.08832830e+05]]

[[1, 1.0, 1.0, 1.0, 1.0, 1.0]]
NUM2
X:[1. 1. 1. 1. 1. 1.]
W:[[  0.7616617  589.78593005 590.1720505  590.22212505 590.47862552
  590.36530333]
 [ -0.68227572 571.86488059 571.96023871 572.54189825 572.23407966
  571.07588235]
 [  0.78601115 541.99891852 542.3708805  543.14380985 541.97910065
  541.99393895]
 [  0.90483172 572.82581494 572.06571533 573.13833238 572.51885867
  573.2602849 ]]

NUM3
X:[1. 1. 1. 1. 1.]
W:[[ 0.38998666 61.34572669 60.40766124


NUM3
X:[1. 1. 1. 1. 1.]
W:[[ 0.38998666 66.83228838 65.89422293 63.53067649 66.07701549]
 [-0.50456679  8.1239361   9.06499011  8.75699094  7.57150297]
 [ 0.43950972 51.78406482 50.87110309 50.73464054 50.96381078]]

NUM4
X:[1. 1. 1. 1.]
W:[[  0.70126531  18.76015968   6.37166124  15.92716039]
 [  0.30062859 -17.94229505  -6.52673317 -16.38742315]]

hidden layer value:[[1, 1.0, 1.0, 1.0, 1.0, 1.0], [1, 1.0, 1.0, 1.0, 1.0], [1, 1.0, 0.9999999999999953, 1.0], [1.0, 2.436856519784998e-18]]
softmax value:[0.7310585786300049, 0.2689414213699951]
cost matrix:[[-14674.600092519811, -14675.090552241836, -14688.279906673533, -14678.597155236306, -14675.813741596403], [-27.631638672353517, -27.29339583357834, -26.67213334826297, -27.230970468297254], [-1.219105046327281, -0.42843177199280674, -1.0733579579018369], [-0.16894142136999513, 0.1689414213699951]]
backpropagation1
(5, 1)
(1, 4)
backpropagation2
(4, 1)
(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
NUM1
X0:[1 1 2 

In [96]:
mlp.output_layer_softmax

[0.7310585786300049, 0.2689414213699951]

In [77]:
mlp.feed_forward()

NUM1
X0:[1 1 2 3 4]
W0:[[-0.16068509  0.50470799  0.46109492  0.03847661  0.81027792]
 [-0.12083023 -0.6812338   0.8447542   0.50466292  0.08030182]
 [-0.54044823  0.58278714  0.65582851  0.87958223  0.12367965]
 [ 0.29138226  0.1752212   0.88864064 -0.640059    0.38742814]
 [-0.59952408  0.72764115 -0.42311419 -0.41275114 -0.24650853]]

[[1, 0.9902699086980399, 0.9383494571856277, 0.9888759693671143, 0.8668535389863096, 0.05009699484478844]]
NUM2
X:[1.         0.99026991 0.93834946 0.98887597 0.86685354 0.05009699]
W:[[-0.6538941  -0.06901708  0.85111539 -0.50101647  0.817482   -0.44366961]
 [ 0.32001684 -0.65636686 -0.57004192  0.72029347  0.24973315 -0.50205472]
 [ 0.66112358 -0.73020662  0.44696989 -0.77567696 -0.18237519  0.69572718]
 [ 0.06496736 -0.3594041  -0.01747826 -0.48758711 -0.84867635  0.27617951]]

NUM3
X:[1.         0.56644751 0.50968616 0.36985215 0.18073941]
W:[[-0.29401495 -0.95321012  0.88070548 -0.55054479  0.12562578]
 [ 0.52565792 -0.62358584  0.34127899 -0.0354

In [78]:
mlp.weights

[[[-0.16068509072388193,
   0.5047079901969567,
   0.46109491734215635,
   0.03847661132287117,
   0.8102779232967972],
  [-0.12083023384972402,
   -0.6812338012911092,
   0.8447541957960134,
   0.5046629229970039,
   0.08030181886772736],
  [-0.5404482280370295,
   0.5827871443204895,
   0.6558285084458031,
   0.8795822340898691,
   0.12367964700894407],
  [0.2913822579691383,
   0.1752212004916256,
   0.8886406405307816,
   -0.6400590026882722,
   0.3874281423962065],
  [-0.5995240750955593,
   0.7276411510446521,
   -0.42311418586755023,
   -0.4127511431331925,
   -0.24650853280150886]],
 [[-0.6538941015993947,
   -0.06901707739673668,
   0.8511153871844348,
   -0.501016474450539,
   0.8174820034979813,
   -0.44366960624924756],
  [0.32001683598615993,
   -0.6563668645492284,
   -0.5700419182150813,
   0.7202934680350248,
   0.24973315208374913,
   -0.5020547205235917],
  [0.6611235821470036,
   -0.7302066225088528,
   0.4469698949992653,
   -0.7756769644281696,
   -0.18237518705787

In [79]:
mlp.weights_without_bias

[[[0.5047079901969567,
   0.46109491734215635,
   0.03847661132287117,
   0.8102779232967972],
  [-0.6812338012911092,
   0.8447541957960134,
   0.5046629229970039,
   0.08030181886772736],
  [0.5827871443204895,
   0.6558285084458031,
   0.8795822340898691,
   0.12367964700894407],
  [0.1752212004916256,
   0.8886406405307816,
   -0.6400590026882722,
   0.3874281423962065],
  [0.7276411510446521,
   -0.42311418586755023,
   -0.4127511431331925,
   -0.24650853280150886]],
 [[-0.06901707739673668,
   0.8511153871844348,
   -0.501016474450539,
   0.8174820034979813,
   -0.44366960624924756],
  [-0.6563668645492284,
   -0.5700419182150813,
   0.7202934680350248,
   0.24973315208374913,
   -0.5020547205235917],
  [-0.7302066225088528,
   0.4469698949992653,
   -0.7756769644281696,
   -0.18237518705787803,
   0.6957271827434017],
  [-0.3594041028655677,
   -0.017478261087138236,
   -0.48758710635958646,
   -0.8486763531231578,
   0.2761795074826874]],
 [[-0.9532101219972584,
   0.8807054831

In [80]:
mlp.back_propagation()

[[-0.5106341264560306, 0.5106341264560307]]
[[-0.5106341264560306, 0.5106341264560307], [0.023275247350912402, 0.08421395110128267, 0.036341135631862136], [-0.01373868444947414, 0.00799279330078951, -0.00787415640857986, -0.009350009873140039], [0.0009503805190520094, -0.003963087219260225, 0.004605980722737955, 2.8360450305398143e-05, -0.0014935448790556517]]
<class 'list'>
[[0.0009503805190520094, -0.003963087219260225, 0.004605980722737955, 2.8360450305398143e-05, -0.0014935448790556517], [-0.01373868444947414, 0.00799279330078951, -0.00787415640857986, -0.009350009873140039], [0.023275247350912402, 0.08421395110128267, 0.036341135631862136], [-0.5106341264560306, 0.5106341264560307]]
backpropagation1
(5, 1)
(1, 4)
backpropagation2
(4, 1)
(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)


In [81]:
mlp.weights_without_bias

[array([[ 0.50461295,  0.46090484,  0.0381915 ,  0.80989777],
        [-0.68083749,  0.84554681,  0.50585185,  0.08188705],
        [ 0.58232655,  0.65490731,  0.87820044,  0.12183725],
        [ 0.17521836,  0.88863497, -0.64006751,  0.3874168 ],
        [ 0.72779051, -0.42281548, -0.41230308, -0.24591111]]),
 array([[-0.06765658,  0.85240456, -0.49965789,  0.81867295, -0.44360078],
        [-0.65715837, -0.57079192,  0.71950308,  0.24904029, -0.50209476],
        [-0.72942687,  0.44770877, -0.77489831, -0.18169261,  0.69576663],
        [-0.3584782 , -0.0166009 , -0.48666251, -0.84786584,  0.27622635]]),
 array([[-0.95452854,  0.87951918, -0.55140563,  0.1252051 ],
        [-0.62835612,  0.33698673, -0.03860161, -0.87435722],
        [ 0.41671542, -0.41863089, -0.46317634,  0.90395494]]),
 array([[ 0.47783971, -0.1695108 ,  0.0408531 ],
        [ 0.62922157,  0.48461437,  0.29034266]])]

In [82]:
mlp.weights

[[[-0.16068509072388193,
   0.5046129521450514,
   0.46090484123834596,
   0.038191497167155565,
   0.8098977710891764],
  [-0.12083023384972402,
   -0.6808374925691832,
   0.8455468132398655,
   0.5058518491627819,
   0.08188705375543145],
  [-0.5404482280370295,
   0.5823265462482157,
   0.6549073123012555,
   0.8782004398730477,
   0.1218372547198489],
  [0.2913822579691383,
   0.17521836444659505,
   0.8886349684407205,
   -0.6400675108233638,
   0.3874167982160843],
  [-0.5995240750955593,
   0.7277905055325576,
   -0.4228154768917391,
   -0.4123030796694758,
   -0.2459111148498866]],
 [[-0.6538941015993947,
   -0.06765657681719549,
   0.8524045558939957,
   -0.4996578889602587,
   0.8186729462215856,
   -0.44360077956884364],
  [0.32001683598615993,
   -0.65715836681845,
   -0.5707919215406005,
   0.7195030799126979,
   0.2490402939678316,
   -0.5020947620160702],
  [0.6611235821470036,
   -0.729426868494073,
   0.4477087660384439,
   -0.7748983080230213,
   -0.18169261302294712,

In [83]:
mlp.feed_forward()

NUM1
X0:[1 1 2 3 4]
W0:[[-0.16068509  0.50461295  0.46090484  0.0381915   0.80989777]
 [-0.12083023 -0.68083749  0.84554681  0.50585185  0.08188705]
 [-0.54044823  0.58232655  0.65490731  0.87820044  0.12183725]
 [ 0.29138226  0.17521836  0.88863497 -0.64006751  0.3874168 ]
 [-0.59952408  0.72779051 -0.42281548 -0.41230308 -0.24591111]]

[[1, 0.9902423983251206, 0.9390336740807804, 0.9887229367029065, 0.866843718719523, 0.05031064641881124]]
NUM2
X:[1.         0.9902424  0.93903367 0.98872294 0.86684372 0.05031065]
W:[[-0.6538941  -0.06765658  0.85240456 -0.49965789  0.81867295 -0.44360078]
 [ 0.32001684 -0.65715837 -0.57079192  0.71950308  0.24904029 -0.50209476]
 [ 0.66112358 -0.72942687  0.44770877 -0.77489831 -0.18169261  0.69576663]
 [ 0.06496736 -0.3584782  -0.0166009  -0.48666251 -0.84786584  0.27622635]]

NUM3
X:[1.         0.56779649 0.50882044 0.37065064 0.18125821]
W:[[-0.29401495 -0.95452854  0.87951918 -0.55140563  0.1252051 ]
 [ 0.52565792 -0.62835612  0.33698673 -0.03860

In [84]:
mlp.back_propagation()

[[-0.5061293744989881, 0.5061293744989881]]
[[-0.5061293744989881, 0.5061293744989881], [0.018543283343290362, 0.07697937345921335, 0.028027329407355923], [-0.012558133447506609, 0.0071560236856377, -0.006324773053861721, -0.009831604111194018], [0.0008462489673560007, -0.003527857523474872, 0.004171917171298427, 0.00020546850866480807, -0.00128381703057129]]
<class 'list'>
[[0.0008462489673560007, -0.003527857523474872, 0.004171917171298427, 0.00020546850866480807, -0.00128381703057129], [-0.012558133447506609, 0.0071560236856377, -0.006324773053861721, -0.009831604111194018], [0.018543283343290362, 0.07697937345921335, 0.028027329407355923], [-0.5061293744989881, 0.5061293744989881]]
backpropagation1
(5, 1)
(1, 4)
backpropagation2
(4, 1)
(1, 5)
backpropagation3
(3, 1)
(1, 4)
backpropagation4
(2, 1)
(1, 3)
